In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from sklearn.datasets import load_iris

In [4]:
X, y = load_iris(return_X_y=True)

In [10]:
len(set(y))

3

In [11]:
num_classes = _

In [12]:
f"We have {y.size} labeled examples across the following {num_classes} classes:\n{set(y)}\n"

'We have 150 labeled examples across the following 3 classes:\n{0, 1, 2}\n'

In [13]:
X[:4], y[:4]

(array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2]]),
 array([0, 0, 0, 0]))

In [14]:
sample_size = int(y.size/4)

In [15]:
X_train, y_train = X[:sample_size], y[:sample_size]

In [16]:
f"Training set labels: {y_train}"

'Training set labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]'

In [17]:
from sklearn.model_selection import train_test_split

In [20]:
np.random.seed(0)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.25)

In [21]:
f"Training set labels: {y_train}"

'Training set labels: [0 2 1 2 1 0 2 0 2 0 0 2 0 2 1 1 1 2 2 1 1 0 1 2 2 0 1 1 1 1 0 0 0 2 1 2 0]'